# Financial Solution Accelerator: Drawing a Company Ecosystem Graph and Analyzing it with Embeddings

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Finance/31.Solution_Company_Ecosystem_Graph_Embeddings.ipynb)



This accelerator will help you process Financial Annual Reports (10K filings) or even Wikipedia data about companies, using John Snow Labs Finance NLP **Named Entity Recognition, Relation Extraction and Assertion Status**, to extract the following information about companies:
- Information about the Company itself (`Trading Symbol`, `State`, `Address`, Contact Information) and other names the Company is known by (`alias`, `former name`).
- People (usually management and C-level) working in that company and their past experiences, including roles and companies
- `Acquisitions` events, including the acquisition dates. `Subsidiaries` mentioned.
- Other Companies mentioned in the report as `competitors`: we will also run a "Competitor check", to understand if another company is just in the ecosystem / supply chain of the company or it is really a competitor
- Temporality (`past`, `present`, `future`) and Certainty (`possible`) of events described, including `Forward-looking statements`.

Also, John Snow Labs provides with offline modules to check for Edgar database (**Entity Linking** to resolve an organization name to its official name and **Chunk Mappers** to map a normalized name to Edgar Database), which are quarterly updated. We will using them to retrieve the `official name of a company`, `former names`, `dates where names where changed`, etc.

The final aim of this accelerator is to help you analyze companies information...

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/im1.png" alt="drawing" width="600"/>

... create a graph...

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/img10.png" alt="drawing" width="800"/>

# Installation


In [ ]:
!pip install johnsnowlabs

In [ ]:
from johnsnowlabs import * 
jsl.install(json_license_path=[YOUR_PATH])

# Starting a session

In [ ]:
spark = jsl.start(json_license_path=[YOUR_PATH])

# Imports

In [ ]:
import os
import sys
import time
import json
import functools 
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import spatial

### Auxiliary Visualization functions 
We will use [NetworkX](https://networkx.org/) to store the graph and [Plotly](https://plotly.com/) to visualize it.

These functions will:
- Use Plotly to visualize a NetworkX graph
- Display relations in a dataframe

In [ ]:
!pip install networkx plotly

Requirement already satisfied: networkx in /databricks/python3/lib/python3.8/site-packages (2.8.8)
Requirement already satisfied: plotly in /databricks/python3/lib/python3.8/site-packages (5.1.0)
Requirement already satisfied: tenacity>=6.2.0 in /databricks/python3/lib/python3.8/site-packages (from plotly) (8.0.1)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from plotly) (1.15.0)
 WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [ ]:
import networkx as nx
G = nx.Graph()

In [ ]:
import plotly.graph_objects as go
import random

def get_nodes_from_graph(graph, pos, node_color):
  """Extracts the nodes from a networkX dataframe in Plotly Scatterplot format"""
  node_x = []
  node_y = []
  texts = []
  hovers = []
  for node in graph.nodes():
    entity = graph.nodes[node]['attr_dict']['entity']
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    texts.append(node)
    hovers.append(entity)

  node_trace = go.Scatter(
    x=node_x, y=node_y, text=texts, hovertext=hovers,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        color=node_color,
        size=40,
        line_width=2))
  
  return node_trace


def get_edges_from_graph(graph, pos, edge_color):
  """Extracts the edges from a networkX dataframe in Plotly Scatterplot format"""
  edge_x = []
  edge_y = []
  hovers = []
  xtext = []
  ytext = []
  for edge in graph.edges():
    relation = graph.edges[edge]['attr_dict']['relation']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    hovers.append(relation)
    xtext.append((x0+x1)/2)
    ytext.append((y0+y1)/2)

  edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color=edge_color),
    mode='lines')
  
  labels_trace = go.Scatter(x=xtext,y= ytext, mode='text',
                              textfont = {'color': edge_color},
                              marker_size=0.5,
                              text=hovers,
                              textposition='top center',
                              hovertemplate='weight: %{text}<extra></extra>')
  return edge_trace, labels_trace


def show_graph_in_plotly(graph, node_color='white', edge_color='grey'):
  """Shows Plotly graph in Databricks"""
  pos = nx.spring_layout(graph)
  node_trace = get_nodes_from_graph(graph, pos, node_color)
  edge_trace, labels_trace = get_edges_from_graph(graph, pos, edge_color)
  fig = go.Figure(data=[edge_trace, node_trace, labels_trace],
               layout=go.Layout(
                  title='Company Ecosystem',
                  titlefont_size=16,                   
                  showlegend=False,
                  width=1600,
                  height=1000,
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers')) 
  fig.show()

In [ ]:
import pandas as pd

def get_relations_df (results, col='relations'):
  """Shows a Dataframe with the relations extracted by Spark NLP"""
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
        rel.result, 
        rel.metadata['entity1'], 
        rel.metadata['entity1_begin'],
        rel.metadata['entity1_end'],
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['entity2_begin'],
        rel.metadata['entity2_end'],
        rel.metadata['chunk2'], 
        rel.metadata['confidence']
    ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df

# Common Componennts
This pipeline will:
1) Split Text into Sentences
2) Split Sentences into Words
3) Use Financial Text Embeddings, trained on SEC documents, to obtain numerical semantic representation of words

These components are common for all the pipelines we will use.

In [ ]:
def get_generic_base_pipeline():
  """Common components used in all pipelines"""
  document_assembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  sentence_detector = nlp.SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")
  
  tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  base_pipeline = Pipeline(stages=[
      document_assembler,
      sentence_detector,
      tokenizer,
      embeddings
  ])

  return base_pipeline
    
generic_base_pipeline = get_generic_base_pipeline()

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

In [ ]:
# Text Classifier
def get_text_classification_pipeline(model):
  """This pipeline allows you to use different classification models to understand if an input text is of a specific class or is something else.
  It will be used to check where the first summary page of SEC10K is, where the sections of Acquisitions and Subsidiaries are, or where in the document
  the management roles and experiences are mentioned"""
  documentAssembler = nlp.DocumentAssembler() \
       .setInputCol("text") \
       .setOutputCol("document")

  useEmbeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

  docClassifier = nlp.ClassifierDLModel.pretrained(model, "en", "finance/models")\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("category")

  nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      useEmbeddings,
      docClassifier])
  
  return nlpPipeline

# Sample Texts from Cadence Design System
Examples taken from publicly available information about Cadence in SEC's Edgar database [here](https://www.sec.gov/Archives/edgar/data/813672/000081367222000012/cdns-20220101.htm) and [Wikipedia](https://en.wikipedia.org/wiki/Cadence_Design_Systems)

In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/cdns-20220101.html.txt

--2022-11-08 16:07:43-- https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/cdns-20220101.html.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347392 (339K) [text/plain]
Saving to: ‘cdns-20220101.html.txt.3’

cdns-20220101.html. 0%[ ] 0 --.-KB/s cdns-20220101.html. 100%[===================>] 339.25K --.-KB/s in 0.004s 

2022-11-08 16:07:43 (89.6 MB/s) - ‘cdns-20220101.html.txt.3’ saved [347392/347392]

In [ ]:
with open('cdns-20220101.html.txt', 'r') as f:
  cadence_sec10k = f.read()
print(cadence_sec10k[:100])

Table of Contents
UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
__________

In [ ]:
pages = [x for x in cadence_sec10k.split("Table of Contents") if x.strip() != '']
print(pages[0])

UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
_____________________________________ 
FORM 10-K 
_____________________________________ 
(Mark One)
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended January 1, 2022 
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _________ to_________.

Commission file number 000-15867 
_____________________________________
 
CADENCE DESIGN SYSTEMS, INC. 
(Exact name of registrant as specified in its charter)
____________________________________ 
Delaware
 
00-0000000
(State or Other Jurisdiction ofIncorporation or Organization)
 
(I.R.S. EmployerIdentification No.)
2655 Seely Avenue, Building 5,
San Jose,
California
 
95134
(Address of Principal Executive Offices)
 
(Zip Code)
(408)
-943-1234 
(Registrant’s Telephone Number, including Area Code) 
Securities registered pursuant to Section 12(b) of the Act:
Title of Each Class
Trading Symbol(s)
Names of Each Exchange on which Registered
Common Stock, $0.01 par value per share
CDNS
Nasdaq Global Select Market
Securities registered pursuant to Section 12(g) of the Act:
None
Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. 
 Yes 
☒
 No 
☐
Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. 
 Yes 
☐ 
No 
☒
Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. 
 Yes 
☒
 No 
☐
Indicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (§ 232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files). 
 Yes 
☒
 No 
☐
Indicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company. See the definitions of “large accelerated filer,” “accelerated filer,” “smaller reporting company,” and “emerging growth company” in Rule 12b-2 of the Exchange Act.
Large Accelerated Filer
☒
Accelerated Filer
☐
Non-accelerated Filer
☐
Smaller Reporting Company
☐
Emerging Growth Company
☐
If an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. 
☐
Indicate by check mark whether the registrant has filed a report on and attestation to its management’s assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its audit report. 
☒
Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act). 
 Yes 
☐ 
No 
☒
The aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold as of the last business day of the registrant’s most recently completed second fiscal quarter ended July 3, 2021 was approximately $38,179,000,000.
On February 5, 2022, approximately 277,336,000 shares of the Registrant’s Common Stock, $0.01 par value, were outstanding.
DOCUMENTS INCORPORATED BY REFERENCE
Portions of the definitive proxy statement for Cadence Design Systems, Inc.’s 2022 Annual Meeting of Stockholders are incorporated by referenc

## Using Text Classification to find Relevant Parts of the Document: 10K Summary
In this case, we know page 0 is always the page with summary information about the company. However, let's suppose we don't know it. We can use Page Classification.

To check the SEC 10K Summary page, we have a specific model called `"finclf_form_10k_summary_item"`

In [ ]:
classification_pipeline = get_text_classification_pipeline('finclf_form_10k_summary_item')
df = spark.createDataFrame([[pages[0]]]).toDF("text")
model = classification_pipeline.fit(df)
result = model.transform(df)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finclf_form_10k_summary_item download started this may take some time.
Approximate size to download 21.3 MB
[ | ][OK!]

In [ ]:
result.select('category.result').show()

+------------------+
 result|
+------------------+
[form_10k_summary]|
+------------------+

Confirmed, page 0 is where the 10K summary is!

## NER: Named Entity Recognition on 10K Summary
Main component to carry out information extraction and extract entities from texts. 

This time we will use a model trained to extract many entities from 10K summaries.

In [ ]:
summary_sample_text = pages[0]

In [ ]:
ner_model_sec10k = finance.NerModel.pretrained("finner_sec_10k_summary", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_summary")

ner_converter_sec10k = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_summary"])\
    .setOutputCol("ner_chunk_sec10k")

summary_pipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_sec10k,
    ner_converter_sec10k
])

finner_sec_10k_summary download started this may take some time.
[ | ][OK!]

Let's visualize the entities with Spark NLP Visualizer

In [ ]:
from sparknlp_display import NerVisualizer

ner_vis = viz.NerVisualizer()

empty_data = spark.createDataFrame([[""]]).toDF("text")

summary_model = summary_pipeline.fit(empty_data)

light_summary_model = LightPipeline(summary_model)

summary_results = light_summary_model.fullAnnotate(summary_sample_text)

In [ ]:
summary_results

Out[291]: [{'ner_summary': [Annotation(named_entity, 1, 6, O, {'word': 'UNITED', 'confidence': '0.9819', 'sentence': '0'}),
 Annotation(named_entity, 8, 13, O, {'word': 'STATES', 'confidence': '0.9875', 'sentence': '0'}),
 Annotation(named_entity, 15, 24, O, {'word': 'SECURITIES', 'confidence': '0.9997', 'sentence': '0'}),
 Annotation(named_entity, 26, 28, O, {'word': 'AND', 'confidence': '0.9754', 'sentence': '0'}),
 Annotation(named_entity, 30, 37, O, {'word': 'EXCHANGE', 'confidence': '0.9886', 'sentence': '0'}),
 Annotation(named_entity, 39, 48, O, {'word': 'COMMISSION', 'confidence': '0.9853', 'sentence': '0'}),
 Annotation(named_entity, 50, 59, O, {'word': 'Washington', 'confidence': '0.9683', 'sentence': '0'}),
 Annotation(named_entity, 60, 60, O, {'word': ',', 'confidence': '0.9622', 'sentence': '0'}),
 Annotation(named_entity, 62, 64, O, {'word': 'D.C', 'confidence': '0.8551', 'sentence': '0'}),
 Annotation(named_entity, 65, 65, O, {'word': '.', 'confidence': '0.9926', 'sentence': '0'}),
 Annotation(named_entity, 67, 71, O, {'word': '20549', 'confidence': '0.9956', 'sentence': '0'}),
 Annotation(named_entity, 73, 109, O, {'word': '_____________________________________', 'confidence': '1.0', 'sentence': '0'}),
 Annotation(named_entity, 112, 115, O, {'word': 'FORM', 'confidence': '0.5657', 'sentence': '0'}),
 Annotation(named_entity, 117, 120, O, {'word': '10-K', 'confidence': '0.9979', 'sentence': '0'}),
 Annotation(named_entity, 123, 159, O, {'word': '_____________________________________', 'confidence': '1.0', 'sentence': '0'}),
 Annotation(named_entity, 163, 163, O, {'word': '(', 'confidence': '1.0', 'sentence': '0'}),
 Annotation(named_entity, 164, 167, O, {'word': 'Mark', 'confidence': '0.9745', 'sentence': '0'}),
 Annotation(named_entity, 169, 171, O, {'word': 'One', 'confidence': '0.9993', 'sentence': '0'}),
 Annotation(named_entity, 172, 172, O, {'word': ')', 'confidence': '1.0', 'sentence': '0'}),
 Annotation(named_entity, 174, 174, O, {'word': '☒', 'confidence': '0.9985', 'sentence': '0'}),
 Annotation(named_entity, 176, 181, O, {'word': 'ANNUAL', 'confidence': '0.9184', 'sentence': '0'}),
 Annotation(named_entity, 183, 188, O, {'word': 'REPORT', 'confidence': '0.9722', 'sentence': '0'}),
 Annotation(named_entity, 190, 197, O, {'word': 'PURSUANT', 'confidence': '0.9883', 'sentence': '0'}),
 Annotation(named_entity, 199, 200, O, {'word': 'TO', 'confidence': '0.9754', 'sentence': '0'}),
 Annotation(named_entity, 202, 208, O, {'word': 'SECTION', 'confidence': '0.9989', 'sentence': '0'}),
 Annotation(named_entity, 210, 211, O, {'word': '13', 'confidence': '0.9787', 'sentence': '0'}),
 Annotation(named_entity, 213, 214, O, {'word': 'OR', 'confidence': '0.9735', 'sentence': '0'}),
 Annotation(named_entity, 216, 217, O, {'word': '15', 'confidence': '0.9941', 'sentence': '0'}),
 Annotation(named_entity, 218, 218, O, {'word': '(', 'confidence': '1.0', 'sentence': '1'}),
 Annotation(named_entity, 219, 219, O, {'word': 'd', 'confidence': '1.0', 'sentence': '1'}),
 Annotation(named_entity, 220, 220, O, {'word': ')', 'confidence': '1.0', 'sentence': '1'}),
 Annotation(named_entity, 222, 223, O, {'word': 'OF', 'confidence': '0.9996', 'sentence': '1'}),
 Annotation(named_entity, 225, 227, O, {'word': 'THE', 'confidence': '0.9985', 'sentence': '1'}),
 Annotation(named_entity, 229, 238, O, {'word': 'SECURITIES', 'confidence': '1.0', 'sentence': '1'}),
 Annotation(named_entity, 240, 247, O, {'word': 'EXCHANGE', 'confidence': '0.9988', 'sentence': '1'}),
 Annotation(named_entity, 249, 251, O, {'word': 'ACT', 'confidence': '0.9991', 'sentence': '1'}),
 Annotation(named_entity, 253, 254, O, {'word': 'OF', 'confidence': '0.999', 'sentence': '1'}),
 Annotation(named_entity, 256, 259, O, {'word': '1934', 'confidence': '0.9999', 'sentence': '1'}),
 Annotation(named_entity, 261, 263, O, {'word': 'For', 'confidence': '0.9993', 'sentence': '1'}),
 Annotation(named_entity, 265, 267, O, {'word': 'the', 'confidence': '1.0', 'sentence': '1'

### Visualize Results

In [ ]:
for r in summary_results:
    displayHTML(ner_vis.display(r, label_col = "ner_chunk_sec10k", document_col = "document", return_html=True))

UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 _____________________________________ FORM 10-K _____________________________________ (Mark One) ☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended January 1, 2022 FISCAL_YEAR OR ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from _________ to_________. Commission file number 000-15867 CFN _____________________________________ CADENCE DESIGN SYSTEMS, INC ORG . (Exact name of registrant as specified in its charter) ____________________________________ Delaware STATE 00-0000000 IRS (State or Other Jurisdiction ofIncorporation or Organization) (I.R.S. EmployerIdentification No.) 2655 Seely Avenue, Building 5, San Jose, California ADDRESS 95134 (Address of Principal Executive Offices) (Zip Code) (408) -943-1234 PHONE (Registrant’s Telephone Number, including Area Code) Securities registered pursuant to Section 12(b) of the Act: Title of Each Class Trading Symbol(s) Names of Each Exchange on which Registered Common Stock TITLE_CLASS , $0.01 TITLE_CLASS_VALUE par value per share CDNS TICKER Nasdaq Global Select Market STOCK_EXCHANGE Securities registered pursuant to Section 12(g) of the Act: None Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. Yes ☒ No ☐ Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. Yes ☐ No ☒ Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. Yes ☒ No ☐ Indicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (§ 232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files). Yes ☒ No ☐ Indicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company. See the definitions of “large accelerated filer,” “accelerated filer,” “smaller reporting company,” and “emerging growth company” in Rule 12b-2 of the Exchange Act. Large Accelerated Filer ☒ Accelerated Filer ☐ Non-accelerated Filer ☐ Smaller Reporting Company ☐ Emerging Growth Company ☐ If an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. ☐ Indicate by check mark whether the registrant has filed a report on and attestation to its management’s assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting firm that prepared or issued its audit report. ☒ Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act). Yes ☐ No ☒ The aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold as of the last business day of the registrant’s most recently completed second fiscal quarter ended July 3, 2021 was approximately $38,179,000,000. On February 5, 2022, approximately 277,336,000 shares of the Registrant’s Common Stock TITLE_CLASS , $0.01 TITLE_CLASS_VALUE par value, were outstanding. DOCUMENTS INCORPORATED BY REFERENCE Portions of the definitive proxy statement for Cadence Desig

## First, let's extract the Organization from NER results

We create a new graph

In [ ]:
G.clear()
G.nodes()

Out[293]: NodeView(())

We extract the organization (entity 'ORG' in the NER results)

In [ ]:
ORG = next(filter(lambda x: x.metadata['entity']=='ORG', summary_results[0]['ner_chunk_sec10k'])).result
ORG

Out[294]: 'CADENCE DESIGN SYSTEMS, INC'

We add our first node to the graph

In [ ]:
# I add our main Organization in the center (x=0, y=0)
G.add_node(ORG, attr_dict={'entity': 'ORG'})

In [ ]:
show_graph_in_plotly(G)

Then, let's add all the summary information from SEC 10K filings (1st page) to that organization.

We can create nodes and add a relation to Cadence directly, since we know it's information of that company.

In [ ]:
for i, r in enumerate(summary_results[0]['ner_chunk_sec10k']):
  text = r.result
  entity = r.metadata['entity']
  
  if entity == 'ORG':
    continue #Already added
  G.add_node(text, attr_dict={'entity': entity}),
  G.add_edge(ORG, text, attr_dict={'relation': 'has_' + entity.lower()})  

In [ ]:
show_graph_in_plotly(G)

## Normalizing the company name to query John Snow Labs datasources for more information about Cadence

Sometimes, companies in texts use a non-official, abbreviated name. For example, we can find `Cadence`, `Cadence Inc`, `Cadence, Inc`, or many other variations, where the official name of the company os `CADENCE DESIGN SYSTEMS INC`, as per registered in SEC Edgar.

Normalizing a company name is super important for data quality purposes. It will help us:
- Standardize the data, improving the quality;
- Carry out additional verifications;
- Join different databases or extract for external sources;

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_edgar_company_name", "en", "finance/models")\
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("normalization")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp = LightPipeline(pipelineModel)

normalized_org = lp.fullAnnotate(ORG)
normalized_org

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finel_edgar_company_name download started this may take some time.
[ | ][OK!]
Out[299]: [{'ner_chunk': [Annotation(document, 0, 26, CADENCE DESIGN SYSTEMS, INC, {})],
 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 26, CADENCE DESIGN SYSTEMS, INC, {'sentence': '0', 'token': 'CADENCE DESIGN SYSTEMS, INC', 'pieceId': '-1', 'isWordStart': 'true'})],
 'normalization': [Annotation(entity, 0, 26, CADENCE DESIGN SYSTEMS INC, {'all_k_results': 'CADENCE DESIGN SYSTEMS INC:::DESIGN WITHIN REACH INC:::AVICI SYSTEMS INC:::HLM DESIGN INC:::NanoWatt Design Inc:::DELTEK SYSTEMS INC:::EPILOG IMAGING SYSTEMS INC', 'all_k_distances': '0.0000:::0.6361:::0.6418:::0.6574:::0.6664:::0.6743:::0.6766', 'confidence': '0.2436', 'all_k_cosine_distances': '0.0000:::0.2023:::0.2060:::0.2161:::0.2220:::0.2273:::0.2289', 'all_k_resolutions': 'CADENCE DESIGN SYSTEMS INC:::DESIGN WITHIN REACH INC:::AVICI SYSTEMS INC:::HLM DESIGN INC:::NanoWatt Design Inc:::DELTEK SYSTEMS INC:::EPILOG IMAGING SYSTEMS INC', 'target_text': 'CADENCE DESIGN SYSTEMS, INC', 'all_k_aux_labels': '770148231:::943314374:::20493372:::562018819:::0:::541252625:::272957582', 'token': 'CADENCE DESIGN SYSTEMS, INC', 'resolved_text': 'CADENCE DESIGN SYSTEMS INC', 'all_k_confidences': '0.2436:::0.1289:::0.1282:::0.1262:::0.1251:::0.1241:::0.1238', 'distance': '0.0000', 'sentence': '0'})]}]

In [ ]:
NORM_ORG = normalized_org[0]['normalization'][0].result
NORM_ORG

Out[300]: 'CADENCE DESIGN SYSTEMS INC'

### NORMALIZED NAME
In Edgar, the company official is different! We need to take it before being able to augment with external information in EDGAR.

- Incorrect: `CADENCE DESIGN SYSTEMS, INC`
- Correct (Official): `CADENCE DESIGN SYSTEMS INC`

In [ ]:
G.add_node(NORM_ORG, attr_dict={'entity': 'ORG'}),
G.add_edge(ORG, NORM_ORG, attr_dict={'relation': 'has_official_name'})  

## DATA AUGMENTATION WITH CHUNK MAPPER

Once we have the normalized name of the company, we can use `John Snow Labs Chunk Mappers`. These are pretrained data sources, which are updated frequently and can be queried inside Spark NLP without sending any API call to any server.

In this case, we will use Edgar Database (`finmapper_edgar_companyname`)

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

chunkAssembler = nlp.Doc2Chunk() \
    .setInputCols("document") \
    .setOutputCol("chunk") \
    .setIsArray(False)

CM = finance.ChunkMapperModel()\
      .pretrained("finmapper_edgar_companyname", "en", "finance/models")\
      .setInputCols(["chunk"])\
      .setOutputCol("mappings")
      
cm_pipeline = Pipeline(stages=[documentAssembler, chunkAssembler, CM])
fit_cm_pipeline = cm_pipeline.fit(empty_data)

df = spark.createDataFrame([[NORM_ORG]]).toDF("text")
r = fit_cm_pipeline.transform(df).collect()

finmapper_edgar_companyname download started this may take some time.
[ | ][OK!]

This is the information we got from that Chunk Mapper about Cadence.

In [ ]:
mappings = r[0]['mappings']
for mapping in mappings:
  text = mapping.result
  relation = mapping.metadata['relation']
  print(f"{ORG} - has_{relation} - {text}")
    
  G.add_node(text, attr_dict={'entity': relation}),
  G.add_edge(ORG, text, attr_dict={'relation': 'has_' + relation.lower()})  

CADENCE DESIGN SYSTEMS, INC - has_name - CADENCE DESIGN SYSTEMS INC
CADENCE DESIGN SYSTEMS, INC - has_sic - SERVICES-PREPACKAGED SOFTWARE [7372]
CADENCE DESIGN SYSTEMS, INC - has_sic_code - 7372
CADENCE DESIGN SYSTEMS, INC - has_irs_number - 770148231
CADENCE DESIGN SYSTEMS, INC - has_fiscal_year_end - 1228
CADENCE DESIGN SYSTEMS, INC - has_state_location - CA
CADENCE DESIGN SYSTEMS, INC - has_state_incorporation - DE
CADENCE DESIGN SYSTEMS, INC - has_business_street - 2655 SEELY AVENUE BLDG 5
CADENCE DESIGN SYSTEMS, INC - has_business_city - SAN JOSE
CADENCE DESIGN SYSTEMS, INC - has_business_state - CA
CADENCE DESIGN SYSTEMS, INC - has_business_zip - 95134
CADENCE DESIGN SYSTEMS, INC - has_business_phone - 4089431234
CADENCE DESIGN SYSTEMS, INC - has_former_name - ECAD INC /DE/
CADENCE DESIGN SYSTEMS, INC - has_former_name_date - 19880609
CADENCE DESIGN SYSTEMS, INC - has_date - 2017-02-10
CADENCE DESIGN SYSTEMS, INC - has_company_id - 813672

In [ ]:
show_graph_in_plotly(G)

## NER and Relation Extraction
NER only extracts isolated entities by itself. But you can combine some NER with specific Relation Extraction Annotators trained for them, to retrieve if the entities are related to each other.

Let's suppose we want to extract information about Acquisitions and Subsidiaries. If we don't know where that information is in the document, we can again use or Text Classifiers to find it.

## Using Text Classification to find Relevant Parts of the Document: Acquisitions and Subsidiaries
To check the SEC 10K Summary page, we have a specific model called `"finclf_acquisitions_item"`

Let's send some pages and check which one(s) contain that information. In a real case, you could send all the pages to the model, but here for time saving purposes, we will show just a subset.

In [ ]:
candidates = [[pages[0]], [pages[1]], [pages[35]], [pages[50]], [pages[67]]] # Some examples

In [ ]:
classification_pipeline = get_text_classification_pipeline('finclf_acquisitions_item')
df = spark.createDataFrame(candidates).toDF("text")
model = classification_pipeline.fit(df)
result = model.transform(df)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finclf_acquisitions_item download started this may take some time.
Approximate size to download 21.5 MB
[ | ][OK!]

In [ ]:
result.select('category.result').show()

+--------------+
 result|
+--------------+
 [other]|
 [other]|
 [other]|
 [other]|
[acquisitions]|
+--------------+

### Acquisitions, Subsidiaries and Former Names
Let's use some NER models to obtain information about Organizations and Dates, and understand if:
- An ORG was acquired by another ORG
- An ORG is a subsidiary of another ORG
- An ORG name is an alias / abbreviation / acronym / etc of another ORG

We will use the deteceted `page[67]` as input

In [ ]:
ner_model_date = finance.NerModel.pretrained("finner_sec_dates", "en", "finance/models")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner_dates")

ner_converter_date = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner_dates"])\
        .setOutputCol("ner_chunk_date")

ner_model_org= finance.NerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner_orgs")

ner_converter_org = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner_orgs"])\
        .setOutputCol("ner_chunk_org")\
        .setWhiteList(['ORG', 'PRODUCT', 'ALIAS'])

chunk_merger = finance.ChunkMergeApproach()\
        .setInputCols('ner_chunk_org', "ner_chunk_date")\
        .setOutputCol('ner_chunk')

pos = PerceptronModel.pretrained()\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos")

dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_filter = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk")\
    .setRelationPairs(["ORG-ORG", "ORG-DATE", "ORG-ROLE", "ROLE-DATE"])\
    .setMaxSyntacticDistance(10)

re_filter_alias = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_alias")\
    .setRelationPairs(["ORG-ALIAS"])\
    .setMaxSyntacticDistance(5)

reDL = finance.RelationExtractionDLModel().pretrained('finre_acquisitions_subsidiaries_md', 'en', 'finance/models')\
    .setInputCols(["re_ner_chunk", "sentence"])\
    .setOutputCol("relations_acq")\
    .setPredictionThreshold(0.1)

reDL_alias = finance.RelationExtractionDLModel()\
    .pretrained("finre_org_prod_alias", "en", "finance/models")\
    .setPredictionThreshold(0.8)\
    .setInputCols(["re_ner_chunk_alias", "sentence"])\
    .setOutputCol("relations_alias")

annotation_merger = finance.AnnotationMerger()\
    .setInputCols("relations_acq", "relations_alias")\
    .setOutputCol("relations")

nlpPipeline = Pipeline(stages=[
        generic_base_pipeline,
        ner_model_date,
        ner_converter_date,
        ner_model_org,
        ner_converter_org,
        chunk_merger,
        pos,
        dependency_parser,
        re_filter,
        re_filter_alias,
        reDL,
        reDL_alias,
        annotation_merger])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(model)

finner_sec_dates download started this may take some time.
[ | ][OK!]
finner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
finre_acquisitions_subsidiaries_md download started this may take some time.
[ | ][OK!]
finre_org_prod_alias download started this may take some time.
[ | ][OK!]

In [ ]:
sample_text = pages[67].replace("“", "\"").replace("”", "\"")

In [ ]:
result = light_model.fullAnnotate(sample_text)
rel_df = get_relations_df(result)

### Visualize Results

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()
displayHTML(re_vis.display(result = result[0], relation_col = "relations", document_col = "document", exclude_relations = ["other", "no_rel"], return_html=True, show_relations=True))

Definite-lived intangible assets acquired with Cadence’s fiscal 2021 acquisitions were as follows: Acquisition Date Fair Value Weighted Average Amortization Period (In thousands) (in years) Existing technology $ 59,100 13.7 years Agreements and relationships 28,900 13.7 years Tradenames, trademarks and patents 4,600 14.3 years Total acquired intangibles with definite lives $ 92,600 13.7 years 2020 Acquisitions In fiscal 2020 DATE , Cadence ORG acquired all of the outstanding equity of AWR Corporation ORG (" AWR ALIAS ") and Integrand Software, Inc ORG . ("Integrand"). These acquisitions enhanced Cadence’s technology portfolio to address growing radio frequency design activity, driven by expanding use of 5G communications. The aggregate cash consideration for these acquisitions was $195.6 million, after taking into account cash acquired of $1.5 million. The total purchase consideration was allocated to the assets acquired and liabilities assumed based on their respective estimated fair values on the acquisition dates. Cadence will also make payments to certain employees, subject to continued employment and other performance-based conditions, through the first quarter of fiscal 2023. With its acquisitions of AWR and Integrand, Cadence recorded $101.3 million of definite-lived intangible assets with a weighted average amortization period of approximately nine years. The definite-lived intangible assets related primarily to existing technology and customer agreements and relationships. Cadence also recorded $119.4 million of goodwill and $25.1 million of net liabilities, consisting primarily of deferred tax liabilities, assumed deferred revenue and trade accounts receivable. The recorded goodwill was primarily related to the acquired assembled workforce and expected synergies from combining operations of the acquired companies with Cadence. None of the goodwill related to the acquisitions of AWR and Integrand is deductible for tax purposes. Cadence ORG completed one additional acquisition during fiscal 2020 DATE that was not material to the consolidated financial statements. Pro Forma Financial Information Cadence has not presented pro forma financial information for any of the businesses it acquired during fiscal 2021 and fiscal 2020 because the results of operations for these businesses are not material to Cadence’s consolidated financial statements. Acquisition-Related Transaction Costs Transaction costs associated with acquisitions, which consist of professional fees and administrative costs, were not material during fiscal 2021, 2020 or 2019 and were expensed as incurred in Cadence’s consolidated income statements. NOTE 7. GOODWILL AND ACQUIRED INTANGIBLES Goodwill The changes in the carrying amount of goodwill during fiscal 2021 and 2020 were as follows: Gross CarryingAmount (In thousands) Balance as of December 28, 2019 $ 661,856 Goodwill resulting from acquisitions 120,564 Effect of foreign currency translation (333) Balance as of January 2, 2021 782,087 Goodwill resulting from acquisitions 154,362 Effect of foreign currency translation (8,091) Balance as of January 1, 2022 $ 928,358 Cadence completed its annual goodwill impairment test during the third quarter of fiscal 2021 and determined that the fair value of Cadence’s single reporting unit exceeded the carrying amount of its net assets and that no impairment existed. 65 was_acquired_by <polyline fill="none" points="104.0,2155.0 105.29591836734693,2153.796079864682 106.59183673469386,2152.5931287133753 107.88775510204083,2151.392115530094 109.18367346938776,2150.1940092988466 110.47959183673468,2148.9997790036464 111.77551020408161,2147.8103936285056 113.07142857142861,2146.626822157435 114.36734693877553,2145.4500335744465 115.66326530612245,2144.280996863552 116.95918367346938,2143.1206810087633 118.25510204081631,2141.9700549940926 119.55102040816324,2140.830087803551 120.8469387755102,2139.7017484211506 122.14285714285715,2138.586005830904 123.43877551020411,2137.4

### Inserting Nodes (Tags) and Relations into a Graph
Now, with entities and Relations connecting them, we can start populating the Graph of the company.

In [ ]:
for t in rel_df.itertuples():
  relation = t.relation
  
  if relation in ['other', 'no_rel']:
    continue
  
  entity1 = t.entity1
  chunk1 = t.chunk1
  entity2 = t.entity2
  chunk2 = t.chunk2
  G.add_node(chunk1,  attr_dict={'entity': entity1})
  G.add_node(chunk2,  attr_dict={'entity': entity2})
  
  G.add_edge(ORG, chunk1, attr_dict={'relation': 'mentions_' + entity1.lower()})  
  G.add_edge(ORG, chunk2, attr_dict={'relation': 'mentions_' + entity2.lower()})  
  
  G.add_edge(chunk1, chunk2, attr_dict={'relation': relation.lower()})  
  

In [ ]:
show_graph_in_plotly(G)

## People's Information
Let's also extract People's name with their current roles and past experiences in other companies (including the dates).

## Using Text Classification to find Relevant Parts of the Document: About Management and their work experience
To check the SEC 10K Summary page, we have a specific model called `"finclf_work_experience_item"`

Let's send some pages and check which one(s) contain that information. In a real case, you could send all the pages to the model, but here for time saving purposes, we will show just a subset.

In [ ]:
candidates = [[pages[4]], [pages[84]], [pages[85]], [pages[86]], [pages[87]]]

In [ ]:
classification_pipeline = get_text_classification_pipeline('finclf_work_experience_item')
df = spark.createDataFrame(candidates).toDF("text")
model = classification_pipeline.fit(df)
result = model.transform(df)
result.select('category.result').show()

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finclf_work_experience_item download started this may take some time.
Approximate size to download 21.5 MB
[ | ][OK!]
+-----------------+
 result|
+-----------------+
 [other]|
 [other]|
 [other]|
[work_experience]|
 [other]|
+-----------------+

**We have some Work Experience in page 86. However, there is 1 sentence hidden in page 4, which is also very relevant.**
However, the model returned `other`. Why?

In [ ]:
pages[4]

Out[508]: '\nThe IC and SoC design excellence requires technologies for custom IC, digital IC design and signoff, and functional verification, and leverages pre-built semiconductor IP. These tools, IP and associated services are specifically designed to meet the growing requirements of engineers designing increasingly complex chips across analog, digital and mixed-signal domains, and perform the associated verification tasks, including validation of low-level software running on the silicon model, thereby enabling design teams to manage complexity and verification throughput without commensurately increasing the team size or extending the project schedule, while reducing technical risks.\nThe second layer of our strategy centers around system innovation. It includes tools and services used for system design of the packages that encapsulate the ICs and the PCBs, system simulation which includes electromagnetic, electro-thermal and other multi-physics analysis necessary as part of optimizing the full system’s performance, radio frequency (“RF”) and microwave systems, and embedded software.\nThe third layer of our strategy addresses pervasive intelligence in new electronics. It starts with providing solutions and services to develop AI-enhanced systems and includes machine learning and deep learning capabilities being added to the Cadence\n®\n technology portfolio to make IP and tools more automated and to produce optimized results faster.\nOur software and emulation products also support cloud access to address the growing computational needs of our customers.\nRecent Acquisitions\nDuring fiscal 2021, we continued to execute our Intelligent System Design strategy and expanded our product offerings and solutions into computational fluid dynamics (“CFD”) with our acquisitions of Belgium-based NUMECA International, a leader in CFD technology, and Pointwise, Inc, a leading provider of CFD meshing technology. The addition of these technologies and talent broadens our System Design and Analysis portfolio and expertise. \nChief Executive Officer Transition\nOn December 15, 2021, Anirudh Devgan assumed the role of President and Chief Executive Officer of Cadence, replacing Lip-Bu Tan. Prior to his role as Chief Executive Officer, Dr. Devgan served as President of Cadence. Concurrently, Mr. Tan transitioned to the role of Executive Chair.\n2\n'

Exploring the page we understand there is a lot of texts about something else which got into the same page. Sometimes, going into a smaller detail may be necessary.

Let's see what happens if we get `paragraphs` instead of `pages.`

In [ ]:
paragraphs = [x for x in pages[4].split('\n') if x.strip() != '']

In [ ]:
paragraphs

Out[519]: ['The IC and SoC design excellence requires technologies for custom IC, digital IC design and signoff, and functional verification, and leverages pre-built semiconductor IP. These tools, IP and associated services are specifically designed to meet the growing requirements of engineers designing increasingly complex chips across analog, digital and mixed-signal domains, and perform the associated verification tasks, including validation of low-level software running on the silicon model, thereby enabling design teams to manage complexity and verification throughput without commensurately increasing the team size or extending the project schedule, while reducing technical risks.',
 'The second layer of our strategy centers around system innovation. It includes tools and services used for system design of the packages that encapsulate the ICs and the PCBs, system simulation which includes electromagnetic, electro-thermal and other multi-physics analysis necessary as part of optimizing the full system’s performance, radio frequency (“RF”) and microwave systems, and embedded software.',
 'The third layer of our strategy addresses pervasive intelligence in new electronics. It starts with providing solutions and services to develop AI-enhanced systems and includes machine learning and deep learning capabilities being added to the Cadence',
 '®',
 ' technology portfolio to make IP and tools more automated and to produce optimized results faster.',
 'Our software and emulation products also support cloud access to address the growing computational needs of our customers.',
 'Recent Acquisitions',
 'During fiscal 2021, we continued to execute our Intelligent System Design strategy and expanded our product offerings and solutions into computational fluid dynamics (“CFD”) with our acquisitions of Belgium-based NUMECA International, a leader in CFD technology, and Pointwise, Inc, a leading provider of CFD meshing technology. The addition of these technologies and talent broadens our System Design and Analysis portfolio and expertise. ',
 'Chief Executive Officer Transition',
 'On December 15, 2021, Anirudh Devgan assumed the role of President and Chief Executive Officer of Cadence, replacing Lip-Bu Tan. Prior to his role as Chief Executive Officer, Dr. Devgan served as President of Cadence. Concurrently, Mr. Tan transitioned to the role of Executive Chair.',
 '2']

In [ ]:
candidates = [[x] for x in paragraphs]

In [ ]:
classification_pipeline = get_text_classification_pipeline('finclf_work_experience_item')
df = spark.createDataFrame(candidates).toDF("text")
model = classification_pipeline.fit(df)
result = model.transform(df)
result.select('category.result').show()

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
finclf_work_experience_item download started this may take some time.
Approximate size to download 21.5 MB
[ | ][OK!]
+-----------------+
 result|
+-----------------+
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
 [other]|
[work_experience]|
[work_experience]|
 [other]|
+-----------------+

**Here we are, if we split in smaller detail (paragraphs, lines), we can found more information than just at page level!**

This is because information in Embeddings gets deluted the bigger the text is. Also, there are some text restrictions (512 tokens in Bert)

In [ ]:
ner_model_role = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_role")

ner_converter_role = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_role"])\
    .setOutputCol("ner_chunk_role")

pos = PerceptronModel.pretrained()\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos")

dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_ner_chunk_filter_role = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk_role", "dependencies"])\
    .setOutputCol("re_ner_chunk_role")\
    .setRelationPairs(["PERSON-ROLE", "ORG-ROLE", "DATE-ROLE"])

re_model_exp = finance.RelationExtractionDLModel.pretrained("finre_work_experience_md", "en", "finance/models")\
    .setInputCols(["re_ner_chunk_role", "sentence"])\
    .setOutputCol("relations")

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_role,
    ner_converter_role,
    pos,
    dependency_parser,
    re_ner_chunk_filter_role,
    re_model_exp,
])


model = nlpPipeline.fit(empty_data)
light_model = LightPipeline(model)

finner_org_per_role_date download started this may take some time.
[ | ][OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
finre_work_experience_md download started this may take some time.
[ | ][OK!]

## Get Results

In [ ]:
sample_text = candidates[9]
sample_text

Out[524]: ['On December 15, 2021, Anirudh Devgan assumed the role of President and Chief Executive Officer of Cadence, replacing Lip-Bu Tan. Prior to his role as Chief Executive Officer, Dr. Devgan served as President of Cadence. Concurrently, Mr. Tan transitioned to the role of Executive Chair.']

In [ ]:
result = light_model.fullAnnotate(sample_text)
rel_df = get_relations_df(result)
rel_df[rel_df["relation"] != "other"]

Out[525]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_role_from,DATE,3,19,"December 15, 2021",ROLE,57,65,President,0.82543
1,has_role_from,DATE,3,19,"December 15, 2021",ROLE,71,93,Chief Executive Officer,0.96655935
2,has_role,PERSON,22,35,Anirudh Devgan,ROLE,57,65,President,0.99948406
3,has_role,PERSON,22,35,Anirudh Devgan,ROLE,71,93,Chief Executive Officer,0.99911696
4,has_role_in_company,ROLE,57,65,President,ORG,98,104,Cadence,0.9898419
5,has_role_in_company,ROLE,71,93,Chief Executive Officer,ORG,98,104,Cadence,0.9961314
6,has_role,ROLE,150,172,Chief Executive Officer,PERSON,175,184,Dr. Devgan,0.98781955
7,has_role_in_company,ROLE,150,172,Chief Executive Officer,ORG,209,215,Cadence,0.9821989
8,has_role,PERSON,175,184,Dr. Devgan,ROLE,196,204,President,0.99987555
9,has_role_in_company,ROLE,196,204,President,ORG,209,215,Cadence,0.9999733


## Visualize Results

In [ ]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()
displayHTML(re_vis.display(result = result[0], relation_col = "relations", document_col = "document", exclude_relations = ["other"], return_html=True, show_relations=True))

On December 15, 2021 DATE , Anirudh Devgan PERSON assumed the role of President ROLE and Chief Executive Officer ROLE of Cadence ORG , replacing Lip-Bu Tan. Prior to his role as Chief Executive Officer ROLE , Dr. Devgan PERSON served as President ROLE of Cadence ORG . Concurrently, Mr. Tan PERSON transitioned to the role of Executive Chair ROLE . <polyline fill="none" points="128.0,155.0 127.79591836734693,154.33673469387753 127.59183673469386,153.6734693877551 127.38775510204083,153.01020408163268 127.18367346938776,152.3469387755102 126.9795918367347,151.68367346938777 126.77551020408161,151.0204081632653 126.5714285714286,150.35714285714292 126.36734693877554,149.69387755102042 126.16326530612245,149.03061224489798 125.95918367346938,148.36734693877554 125.75510204081633,147.70408163265307 125.55102040816325,147.0408163265306 125.34693877551021,146.3775510204082 125.14285714285715,145.71428571428572 124.93877551020411,145.05102040816328 124.73469387755104,144.38775510204084 124.53061224489798,143.7244897959184 124.32653061224491,143.0612244897959 124.12244897959185,142.3979591836735 123.91836734693877,141.734693877551 123.7142857142857,141.07142857142856 123.51020408163266,140.40816326530614 123.3061224489796,139.74489795918367 123.10204081632655,139.08163265306123 122.89795918367348,138.41836734693877 122.6938775510204,137.75510204081633 122.48979591836736,137.09183673469389 122.2857142857143,136.42857142857144 122.08163265306123,135.765306122449 121.87755102040816,135.10204081632654 121.67346938775512,134.4387755102041 121.46938775510205,133.77551020408163 121.26530612244899,133.11224489795921 121.06122448979592,132.44897959183675 120.85714285714286,131.7857142857143 120.6530612244898,131.12244897959184 120.44897959183675,130.4591836734694 120.24489795918367,129.79591836734693 120.04081632653062,129.13265306122452 119.83673469387756,128.46938775510205 119.63265306122449,127.80612244897961 119.42857142857144,127.14285714285717 119.22448979591836,126.4795918367347 119.0204081632653,125.81632653061226 118.81632653061224,125.15306122448982 118.61224489795919,124.48979591836735 118.40816326530611,123.8265306122449 118.20408163265306,123.16326530612245 118.0,122.5 118.0,122.5 117.79591836734691,121.83673469387755 117.59183673469387,121.17346938775509 117.38775510204083,120.51020408163266 117.18367346938776,119.84693877551021 116.97959183673468,119.18367346938776 116.77551020408161,118.5204081632653 116.57142857142861,117.85714285714289 116.36734693877553,117.19387755102042 116.16326530612247,116.53061224489795 115.95918367346937,115.86734693877548 115.75510204081631,115.20408163265304 115.55102040816324,114.54081632653057 115.3469387755102,113.87755102040816 115.14285714285715,113.21428571428571 114.93877551020408,112.55102040816328 114.73469387755102,111.88775510204083 114.53061224489797,111.22448979591837 114.3265306122449,110.56122448979592 114.12244897959184,109.89795918367346 113.91836734693875,109.23469387755101 113.7142857142857,108.57142857142856 113.51020408163266,107.90816326530613 113.30612244897958,107.24489795918367 113.10204081632652,106.58163265306122 112.89795918367346,105.91836734693875 112.6938775510204,105.25510204081633 112.48979591836735,104.59183673469387 112.28571428571429,103.92857142857144 112.08163265306123,103.26530612244898 111.87755102040816,102.60204081632654 111.6734693877551,101.9387755102041 111.46938775510205,101.27551020408163 111.26530612244898,100.61224489795917 111.06122448979592,99.94897959183672 110.85714285714286,99.28571428571428 110.65306122448979,98.62244897959184 110.44897959183675,97.9591836734694 110.24489795918367,97.29591836734693 110.0408163265306,96.63265306122449 109.83673469387756,95.96938775510205 109.63265306122449,95.3061224489796 109.42857142857144,94.64285714285714 109.22448979591836,93.9795918367347 109.0204081632653,93.31632653061224 108.81632653061224,92.6530612244898 108.61224489795919,91.98979591836735 108.40816326530613,91.3265306122449 108.20408163265306,90.663265

## Adding to graph

In [ ]:
for t in rel_df.itertuples():
  relation = t.relation
  if relation == 'other':
    continue
  entity1 = t.entity1
  chunk1 = t.chunk1
  entity2 = t.entity2
  chunk2 = t.chunk2
  G.add_node(chunk1,  attr_dict={'entity': entity1})
  G.add_node(chunk2,  attr_dict={'entity': entity2})
  
  G.add_edge(ORG, chunk1, attr_dict={'relation': 'mentions_' + entity1.lower()})  
  G.add_edge(ORG, chunk2, attr_dict={'relation': 'mentions_' + entity2.lower()})  
  
  G.add_edge(chunk1, chunk2, attr_dict={'relation': relation.lower()})  
  

In [ ]:
show_graph_in_plotly(G)

# Understanding the context of mentioned companies to identify COMPETITORS
Many Companies may be mentioned in the report. Most of them are just organizations in the ecosystem of the Cadence. Others, may be competitors.

We can analyze the surrounding context of the extracted `ORG` to check if they are competitors or not.

In [ ]:
ner = finance.NerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(['ORG', 'PRODUCT'])

assertion = finance.AssertionDLModel.pretrained("finassertion_competitors", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"])\
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner,
    ner_converter,
    assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(model)

finner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
finassertion_competitors download started this may take some time.
[ | ][ / ][ — ][ \ ][OK!]

### Get Results

In [ ]:
sample_text = ["""In the rapidly evolving market, certain elements of our application compete with Microsoft, Google, InFocus, Bluescape, Mersive, Barco, Nureva and Prysm. But, Oracle  and IBM are out of our league."""]

chunks=[]
entities=[]
status=[]


light_result = light_model.fullAnnotate(sample_text)[0]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [ ]:
df

Out[531]:

,chunks,entities,assertion
0,Microsoft,ORG,COMPETITOR
1,Google,ORG,COMPETITOR
2,InFocus,ORG,COMPETITOR
3,Bluescape,ORG,COMPETITOR
4,Mersive,ORG,COMPETITOR
5,Barco,ORG,COMPETITOR
6,Nureva,ORG,COMPETITOR
7,Prysm,ORG,COMPETITOR
8,Oracle,ORG,NO_COMPETITOR
9,IBM,ORG,NO_COMPETITOR


### Visualize Assertion Result

In [ ]:
vis = viz.AssertionVisualizer()

vis.set_label_colors({'COMPETITOR':'#008080', 'NO_COMPETITOR':'#800080'})
    
light_result = light_model.fullAnnotate(sample_text)[0]

displayHTML(vis.display(light_result, 'ner_chunk', 'assertion', return_html=True))


In the rapidly evolving market, certain elements of our application compete with Microsoft ORG COMPETITOR , Google ORG COMPETITOR , InFocus ORG COMPETITOR , Bluescape ORG COMPETITOR , Mersive ORG COMPETITOR , Barco ORG COMPETITOR , Nureva ORG COMPETITOR and Prysm ORG COMPETITOR . But, Oracle ORG NO_COMPETITOR and IBM ORG NO_COMPETITOR are out of our league.

### Adding it to the graph

In [ ]:
for t in df.itertuples():
  chunks = t.chunks
  entities = t.entities
  assertion = t.assertion

  G.add_node(chunks,  attr_dict={'entity': entities})
  
  G.add_edge(ORG, chunks, attr_dict={'relation': 'is_' + assertion.lower()})
  

In [ ]:
show_graph_in_plotly(G)

# Annex 1: Detecting Temporality and Certainty in Affirmations

In [ ]:
ner_model_role = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_role")

ner_converter_role = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_role"])\
    .setOutputCol("ner_chunk")

assertion = finance.AssertionDLModel.pretrained("finassertion_time", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setMaxSentLen(1200)

assertion_pipeline = Pipeline(stages=[
    generic_base_pipeline,
    ner_model_role,
    ner_converter_role,
    assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

assertion_model = assertion_pipeline.fit(empty_data)

light_model_assertion = LightPipeline(assertion_model)

finner_org_per_role_date download started this may take some time.
[ | ][OK!]
finassertion_time download started this may take some time.
[ | ][ / ][ — ][OK!]

### Get Result

In [ ]:
sample_text = ["""Joseph Costello was the CEO of the company since founded in 1988 until 1997. He was followed by Lip-Bu Tan for the 2009–2021 period. Currently, Anirudh Devgan is the CEO since 2021""",
              
              """In 2007, Cadence was rumored to be in talks with Kohlberg Kravis Roberts and Blackstone Group regarding a possible sale of the company.""",
              """In 2008, Cadence withdrew a $1.6 billion offer to purchase rival Mentor Graphics.""",
              
               """ The Cadence Giving Foundation will also support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and STEM education.""",
              """This stand-alone, non-profit foundation will partner with other charitable initiatives to support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and science, technology, engineering, and mathematics (“STEM”) education""",
              
              """Cadence employees could purchase common stock at a price equal to 85% of the lower of the fair market value at the beginning or the end of the applicable offering period"""]

chunks=[]               
entities=[]
status=[]

light_results = light_model_assertion.fullAnnotate(sample_text)

for light_result in light_results:
  for n,m in zip(light_result['ner_chunk'], light_result['assertion']):
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [ ]:
df

Out[539]:

,chunks,entities,assertion
0,Joseph Costello,PERSON,PAST
1,CEO,ROLE,PAST
2,1988,DATE,PAST
3,1997,DATE,PAST
4,2009–2021,DATE,PAST
5,Anirudh Devgan,PERSON,PRESENT
6,CEO,ROLE,PRESENT
7,2021,DATE,PRESENT
8,2007,DATE,PAST
9,Cadence,ORG,PAST


### Visualize Assertion Result

In [ ]:
vis = viz.AssertionVisualizer()

for light_result in light_results:
  displayHTML(vis.display(light_result, 'ner_chunk', 'assertion', return_html=True))


Joseph Costello PERSON PAST was the CEO ROLE PAST of the company since founded in 1988 DATE PAST until 1997 DATE PAST . He was followed by Lip-Bu Tan for the 2009–2021 DATE PAST period. Currently, Anirudh Devgan PERSON PRESENT is the CEO ROLE PRESENT since 2021 DATE PRESENT

In 2007 DATE PAST , Cadence ORG PAST was rumored to be in talks with Kohlberg Kravis Roberts and Blackstone Group ORG PAST regarding a possible sale of the company.

In 2008 DATE PAST , Cadence ORG PAST withdrew a $1.6 billion offer to purchase rival Mentor Graphics ORG PAST .

The Cadence Giving Foundation ORG FUTURE will also support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and STEM education.

This stand-alone, non-profit foundation will partner with other charitable initiatives to support critical needs in areas such as diversity, equity and inclusion, environmental sustainability and science, technology, engineering, and mathematics (“STEM”) education

Cadence ORG POSSIBLE employees could purchase common stock at a price equal to 85% of the lower of the fair market value at the beginning or the end of the applicable offering period

# About Graph Embeddings
We got an example graph of a company. We can continue processing SEC and Wikidata information and populating the graph to get a better understanding of the company’s ecosystem.

After we are happy with the information contained in it, we can use Graph Embeddings to:

- Obtain a numerical representation of the company’s ecosystem;
- Be able to compare company graphs and check for similarity between companies. For example, for competition analysis.
- Be able to compare specific nodes or specific edges and check for similarity between them. For example, for new link prediction.


In [ ]:
!pip install node2vec

## Node Embeddings

In [ ]:
# pip install node2vec
from node2vec import Node2Vec

# Generate walks
node2vec = Node2Vec(G, dimensions=20, walk_length=16, num_walks=100)

In [ ]:
# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

In [ ]:
model.wv.get_vector('AWR')

Out[52]: array([-0.02652522, -0.27153963, 0.08497199, -0.19858257, -0.04536055,
 0.08251562, 0.8951953 , 0.31521553, -0.7170699 , 0.26028967,
 0.52876234, 0.11620306, 0.07119451, -0.308913 , -0.01593829,
 0.0336991 , 0.04910186, -0.2391716 , -0.29850596, -0.46639618],
 dtype=float32)

In [ ]:
for node, _ in model.wv.most_similar('AWR'):
    print(node)

January 15, 2020
AWR Corporation
Delaware
Cadence
95134
CA
January 1, 2022
1228
CDNS
Nureva

## Edge Embeddings

In [ ]:
from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

In [ ]:
edges_embs[('AWR', 'AWR Corporation')]

Out[55]: array([ 6.0157860e-03, -8.6146491e-03, 1.1243380e-02, 1.1806695e-01,
 1.1654351e-02, -5.8547879e-04, 6.0034513e-01, 9.8731108e-02,
 1.7592399e-01, 6.0749445e-02, 3.9436898e-01, 2.4251498e-02,
 1.5226916e-03, 1.4180225e-01, -5.5486225e-03, 6.7649889e-03,
 6.0799732e-03, 7.8225002e-02, 1.0639255e-01, 4.1596767e-01],
 dtype=float32)

In [ ]:
edges_kv = edges_embs.as_keyed_vectors()
edges_kv.most_similar(str(('AWR', 'AWR Corporation')))

Generating edge features: 0%| | 0/1326.0 [00:00<?, ?it/s]Generating edge features: 100%|██████████| 1326/1326.0 [00:00<00:00, 146432.35it/s]
Out[56]: [("('AWR Corporation', 'January 15, 2020')", 0.9908008575439453),
 ("('AWR', 'February 6, 2020')", 0.9033723473548889),
 ("('AWR Corporation', 'February 6, 2020')", 0.9018533229827881),
 ("('AWR Corporation', 'AWR Corporation')", 0.9008990526199341),
 ("('AWR Corporation', 'Cadence')", 0.9003079533576965),
 ("('January 15, 2020', 'January 15, 2020')", 0.8977513909339905),
 ("('AWR', 'Integrand Software Inc')", 0.8970919847488403),
 ("('AWR', 'January 15, 2020')", 0.8962301015853882),
 ("('AWR', 'AWR')", 0.8904980421066284),
 ("('Integrand Software Inc', 'January 15, 2020')", 0.8856679797172546)]

## Want to go further and analyse subgraph Embeddings, to compare, for example, different Companies subgraphs?
Check `https://github.com/benedekrozemberczki/graph2vec` or get in touch with us at support@johnsnowlabs.com or in our [Slack](https://www.johnsnowlabs.com/slack-redirect/), #finance channel.